In [6]:
# 基于用户的协同过滤推荐算法
import os
import random
import math
from operator import itemgetter
import shelve
from contextlib import closing
import pandas as pd
import numpy as np
from datetime import datetime
import glob

In [ ]:
n_sim_user = 20
n_rec_movie = 10

trainSet = {}
testSet = {}
    
user_sim_matrix = {}
movie_count = 0

In [3]:
# 读取文件
path = r"ratings.csv"
# glob.glob(path+'\*.csv')

def load_data_from_csv(path):
    # 当用pd.read_csv( )出现 OSError: Initializing from file failed 问题时，一般是因为参数文件名或文件路径包含中文，
    # 所以改为 fr = open(path) data = pd.read_csv(fr)
    # 或者  pd.read_csv(path, engine='python')
    if os.path.exists(path):
        data = pd.read_csv(path, engine='python',header=0)
#         with open (path) as f:
#             data = pd.read_csv(f)
    return data
data = load_data_from_csv(path)
# data = load_data_from_csv(path)
data


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


In [4]:
np.savez('ratings_csv_read.npz',data)

array([[1.00000000e+00, 1.00000000e+00, 4.00000000e+00, 9.64982703e+08],
       [1.00000000e+00, 3.00000000e+00, 4.00000000e+00, 9.64981247e+08],
       [1.00000000e+00, 6.00000000e+00, 4.00000000e+00, 9.64982224e+08],
       ...,
       [6.10000000e+02, 1.68250000e+05, 5.00000000e+00, 1.49427305e+09],
       [6.10000000e+02, 1.68252000e+05, 5.00000000e+00, 1.49384635e+09],
       [6.10000000e+02, 1.70875000e+05, 3.00000000e+00, 1.49384642e+09]])

In [5]:
data.loc[0]

userId               1.0
movieId              1.0
rating               4.0
timestamp    964982703.0
Name: 0, dtype: float64

In [ ]:
# data = np.load('ratings.npz')
# df = data['arr_0']
# # pd.DataFrame(df)
# df[]
(data.loc[0])

In [4]:
# 原始方法 read file 
def load_file(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i == 0:
                # 去掉文件第一行的title
                continue
            yield line.strip('\r\n')
    print('Load %s success!' % filename)

def get_dataset(filename, pivot=0.75):
    trainSet = {}
    testSet = {}
    trainSet_len = 0
    testSet_len = 0
    # 加载文件， 按行读取
    for line in load_file(filename):
        # 读取列属性
        user, movie, rating, timestamp = line.split(',')
        user = int(user)
        movie = int(movie)
        rating = float(rating)
        # 数据划分测试集合和数据集合 (0,1) < (0,pivot)
        if random.random() < pivot:
            trainSet.setdefault(user, {})
            trainSet[user][movie] = rating
            trainSet_len += 1
        else:
            testSet.setdefault(user, {})
            testSet[user][movie] = rating
            testSet_len += 1
    print('Split trainingSet and testSet success!')
    print('user - movie - matrix')
    print('TrainSet = %s' % trainSet_len)
    print('TestSet = %s' % testSet_len)
    return trainSet, testSet


In [7]:
path = r"ratings.csv"
trainSet, testSet = get_dataset(path)

Load ratings.csv success!
Split trainingSet and testSet success!
user - movie - matrix
TrainSet = 75699
TestSet = 25137


数据处理 

user movie 


In [8]:
def count_user(trainSet):
    # 统计电影被看的次数
    movie_popular = {}
    user_list = {}
    for user, movies in trainSet.items():
        if user not in user_list:
            user_list.setdefault(user, 0)
        for movie in movies:
            if movie not in movie_popular:
                movie_popular[movie] = 0
            movie_popular[movie] += 1
    
    print("Total user number = %d" % len(user_list))
    print("Total movie number = %d" % len(movie_popular))
    return movie_popular, user_list

movie_popular, user_list = count_user(trainSet)

Total user number = 610
Total movie number = 8757


In [9]:
len(user_list)

610

In [ ]:
def sim_movie(trainSet, movie_popular):
    sim = tl.Similarity()
    movie_sim_matrix = {}
    i = 0
    max = len(movie_popular)
    # 遍历训练数据，获得用户对有过的行为的物品

    movie_list = list(movie_popular.keys())

    for i1 in range(0, len(movie_list)):
        # 遍历该用户每件物品项
        for i2 in range(i1, len(movie_list)):
            # 遍历该用户每件物品项
            
            m1, m2 = movie_list[i1], movie_list[i2]

            movie_sim_matrix.setdefault(m1, {})
            movie_sim_matrix.setdefault(m2, {})
            if m1 == m2:
                movie_sim_matrix[m1].setdefault(m2,1)
                continue 
                
            u1 = trainSet.get(m1)
            u2 = trainSet.get(m2)
            
            simi = sim.cosine_similarity(u1,u2)

            movie_sim_matrix[m1].setdefault(m2, simi)
            movie_sim_matrix[m2].setdefault(m1, simi)
        i += 1
        print("\r{0}%".format(round(i/max*100,4)), end='')
        
    print()
    # movie and movie 矩阵
    print("Build 同现矩阵co-rated users matrix success!")
    return movie_sim_matrix

In [ ]:
path = r"F:\毕业设计\ItemCF和UserCF协同过滤算法参考代码\MovieRecommendation-master\UserCF" 
glob.glob(path+'\*data*')
from contextlib import closing
import shelve

def data_init(path):
    #self.trainSet, self.testSet = load_data.get_dataset(path)
    #  读取测试数据 
    with closing(shelve.open(path,'r')) as shelf:
        trainSet = shelf['train']
        testSet = shelf['test']

    print("train data",len(trainSet),"- test data",len(testSet))
    return trainSet, testSet
trainSet, testSet = data_init(path+'\\time0data')


In [ ]:
a = pd.Series(trainSet['1'])
b = pd.Series(trainSet['2'])
print(type(a),type(b))


In [ ]:
pd.DataFrame(trainSet)

In [ ]:
def sim_by_series(a,b):
    mul_a_b = a.astype(np.float).mul(b.astype(np.float), fill_value=0).sum()
    a2 = np.sqrt(np.sum(a.astype(np.float).mul(a.astype(np.float),fill_value=0)))
    b2 = np.sqrt(np.sum(b.astype(np.float).mul(b.astype(np.float),fill_value=0)))
    return mul_a_b / (a2 * b2)

# 余弦相似度
def cosine(x,y):
    x = pd.Series(x)
    y = pd.Series(y) 
    xy = x.astype(float).mul(y.astype(float), fill_value=0)
    x = np.linalg.norm(x.values.astype(float),2)
    y = np.linalg.norm(y.values.astype(float),2)
    return xy.sum() / (x*y)


cosine(trainSet.get('3'),trainSet.get('2'))


In [ ]:
# 计算电影之间的相似度, 利用其他算法
def calc_movie_sim_test(trainSet, func):
    movie_sim_matrix = {}

    for user1 in trainSet.keys():
        movie_sim_matrix.setdefault(user1, {})
        for user2 in trainSet.keys():
            if user1 == user2:
                movie_sim_matrix.get(user1).setdefault(user2, 1)
            else:
                movie_sim_matrix.get(user1).setdefault(user2, func(trainSet.get(user1), trainSet.get(user2)))
    # movie{1,2}=sim
    return  movie_sim_matrix


In [ ]:
func = cosine
pd.DataFrame(calc_movie_sim_test(trainSet, func))

In [ ]:
sim_user = np.array(np.zeros((1000,1000)))

for i in trainSet.keys():
    a = pd.Series(trainSet[i])
    for j in trainSet.keys():
#         print(i,j)
        b = pd.Series(trainSet[j])
        sim_user[int(i),int(j)] = sim_by_series(a,b)
        pass
sim_user

In [ ]:
pd.DataFrame(sim_user)

In [ ]:
np.sum(mul_a_b) / (a2*b2)

In [ ]:
sim_user = np.array(np.zeros((10,10)))
print(sim_user)

In [ ]:
import sys,time
for i in range(100):
    sys.stdout.write(‘\r%s%%‘%(i+1))
    sys.stdout.flush()
    time.sleep(0.1)